# Topic Selection and Postprocessing

In [1]:
%matplotlib inline
import pickle
from os import listdir, makedirs
from os.path import join, isfile, exists, dirname
import gc
import re
import math

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns
%pylab inline
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import TfidfModel, LdaModel
from gensim.models import Word2Vec, Doc2Vec, FastText
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from tqdm._tqdm_notebook import tqdm_notebook

from constants import *
from utils import load, init_logging, tprint, TopicsLoader, Unlemmatizer
from topic_reranking import Reranker

tqdm_notebook.pandas()

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.options.display.max_columns = 80
pd.options.display.max_rows = 2000
pd.options.display.precision = 3

-----

## Joining Topics / Labels / Scores

In [3]:
phra = load('phrases', 'lemmap')
wikt = load('wikt', 'lemmap')

Loading ../data/preprocessed/dewiki_phrases_lemmatization_map.pickle


In [4]:
args = ['e42', 100]
metrics = ['c_npmi_wikt', 'pairwise_similarity_ftx']

def combine(dataset, min_regular_terms=6, min_cnpmi=0, min_pairsim=0.3):
    topics = load(dataset, 'topics', *args)
    labels_w2v = load(dataset, 'labels', *args, 'w2v', 'minimal').rename(columns=lambda x: x + '_w2v').reset_index(level='label_method', drop=True)
    labels_ftx = load(dataset, 'labels', *args, 'ftx', 'minimal').rename(columns=lambda x: x + '_ftx').reset_index(level='label_method', drop=True)
    scores = load(dataset, 'scores', *args, 'ftx', 'minimal')[metrics]

    scores['rank_cnpmi'] = scores.c_npmi_wikt.rank()
    scores['rank_pairsim'] = scores.pairwise_similarity_ftx.rank()
    scores['rank_mean'] = (scores.rank_cnpmi + scores.rank_pairsim) / 2
    col_mean = scores[metrics].mean()
    col_stdv = scores[metrics].std()
    print(col_mean)
    scores_norm = (scores[metrics] - col_mean) / col_stdv
    scores_norm = scores_norm.rename(columns=lambda x: x + '_norm')
    scores_norm['avg'] = scores_norm.mean(axis=1)
    scores = scores.join(scores_norm)
    scores['avg_rank'] = scores.avg.rank()
    print('average difference per topic between the mean of the individual ranks and the rank of the normalized mean:', (scores.rank_mean - scores.avg_rank).abs().sum() / len(scores))
    # an advantage of ranking by taking the normalized mean: minimal chance of equal ranks -> disadvantage: metric is not comparable to other datasets anymore -> use metrics as a filter

    df = pd.concat([topics, labels_w2v, labels_ftx, scores], axis=1)
    df['count_regular_terms'] = df.loc[:, 'term0':'term9'].applymap(lambda x: x in wikt.index).sum(axis=1)
    df = df.query('count_regular_terms >= @min_regular_terms and c_npmi_wikt >= @min_cnpmi and pairwise_similarity_ftx >= @min_pairsim')
    print(len(df))
    df = df.sort_values('avg_rank', ascending=False)
    return df

In [5]:
dfd = combine('dewac', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfn = combine('n', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfs = combine('s', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfo = combine('o', min_regular_terms=6, min_cnpmi=-0.1, min_pairsim=0.3)

Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/dewac_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/dewac_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/dewac_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.123
pairwise_similarity_ftx    0.457
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.12
72
Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/news_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/news_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/news_noun_bow_label-candidates.csv
Readin

In [12]:
dfp = combine('p', min_regular_terms=7, min_cnpmi=0.05, min_pairsim=0.3)
dfp

Loading dictionary from ../data/preprocessed/LDAmodel/noun/bow/PoliticalSpeeches_noun_bow.dict
Loading model from ../data/preprocessed/LDAmodel/noun/bow/e42/PoliticalSpeeches_LDAmodel_e42_100_ep30
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/PoliticalSpeeches_noun_bow_label-candidates.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/PoliticalSpeeches_noun_bow_label-candidates_ftx.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/PoliticalSpeeches_noun_bow_topic-scores.csv
Reading ../data/preprocessed/LDAmodel/noun/bow/topics/PoliticalSpeeches_noun_bow_topic-scores_germanet.csv
c_npmi_wikt                0.050
pairwise_similarity_ftx    0.374
dtype: float64
average difference per topic between the mean of the individual ranks and the rank of the normalized mean: 2.26
43


term0  \
dataset           param_id nb_topics topic_idx                       
PoliticalSpeeches e42      100       32                      Opfer   
                                     62                       Film   
                                     11                Universität   
                                     75                  Abrüstung   
                                     14                     Schule   
                                     72               Gewerkschaft   
                                     84         erneuerbar_Energie   
                                     0                Wissenschaft   
                                     65                       Buch   
                                     38                   Freiheit   
                                     53                Ausstellung   
                                     22                 Erinnerung   
                                     71                    Bildung   
                                     66                     Kultur   
                                     78                 Demokratie   
                                     7                    Religion   
                                     18                      Krise   
                                     59              Menschenrecht   
                                     74                Vertreibung   
                                     47                 Sicherheit   
                                     81           europäisch_Union   
                                     58                   Stiftung   
                                     55                    Politik   
                                     54                     Kultur   
                                     82                       OSZE   
                                     35                Unternehmen   
                                     25                      Stadt   
                                     95               Gesellschaft   
                                     31                  Sparkasse   
                                     5                       Musik   
                                     23                       Bund   
                                     21             Zusammenarbeit   
                                     86                      Media   
                                     69                     Syrien   
                                     60                   Internet   
                                     30                     Israel   
                                     24                  Parlament   
                                     33                       Idee   
                                     41             Auswärtige_Amt   
                                     45                     Kosovo   
                                     85                      China   
                                     19                         EU   
                                     46                       Jude   

                                                           term1  \
dataset           param_id nb_topics topic_idx                     
PoliticalSpeeches e42      100       32               Erinnerung   
                                     62                     Kino   
                                     11               Hochschule   
                                     75                     Iran   
                                     14                  Schüler   
                                     72              Unternehmen   
                                     84                     Ziel   
                                     0                 Forschung   
                                     65                Literatur   
                                     38                    Recht   
                                     53                    Kunst   
                          

In [13]:
df = pd.concat([dfd, dfn, dfp, dfo]).loc[:, 'term0':'label19_w2v'].rename(columns=lambda x: x.replace('_w2v', '')).reset_index(level=['param_id', 'nb_topics'], drop=True).sort_index()
df

term0             term1  \
dataset             topic_idx                                         
OnlineParticipation 1                      Arbeit      Gesellschaft   
                    2                   Vorschlag    Bürgerhaushalt   
                    3                       Seite            Brücke   
                    4                        Köln             Stadt   
                    6                        Idee             Stadt   
                    7                     Nutzung         Ehrenfeld   
                    8                     Schüler         Schülerin   
                    11                    Gebäude             Miete   
                    13                 Schlagloch              Zone   
                    14                     Lösung          Eindruck   
                    15                   Haushalt       Investition   
                    16                       Bonn              Oper   
                    17                 Stadt_Köln           Mülheim   
                    19                     Radweg               Weg   
                    20                Deutschland              Welt   
                    21                Unternehmen             Kunst   
                    23                       Euro              Jahr   
                    25                      Stadt           Angebot   
                    26                       Hund         Kontrolle   
                    28                      Platz              Müll   
                    34                        EUR          Leistung   
                    35                 Moderation              Gruß   
                    37                        PKW        Verbindung   
                    38                    Beitrag              Dank   
                    42                       Kind            Schule   
                    49                     Verein              Jahr   
                    50                    Wohnung          Wohnraum   
                    54                        Ort           Vorbild   
                    55                   Programm               Job   
                    56                     Antrag            Schutz   
                    59              Beschilderung             Recht   
                    60                     Schild             Linie   
                    62                 Spielplatz    Straßenlaterne   
                    64                   Übergang     Verkehrsinsel   
                    65                    Bahnhof        Kölnstraße   
                    66              Zebrastreifen              Höhe   
                    68              Fahrradfahrer     Einbahnstraße   
                    72                       Geld           Meinung   
                    74                        Bad        Grundstück   
                    79                       Auto               Rad   
                    81                     Fläche              Frau   
                    82                       ÖPNV            Stunde   
                    83                        Rat        Verwaltung   
                    88                     Anfang            Winter   
                    89                       Bahn        Stellplatz   
                    90                  Radfahrer         Fußgänger   
                    92                        Bus       Haltestelle   
                    94                 Autofahrer         Radfahrer   
                    95                       Haus               Uhr   
                    96                   Richtung            Radweg   
                    97                     Gehweg              Park   
PoliticalSpeeches   0                Wissenschaft         Forschung   
                    5                       Musik             Abend   
                    7                    Religion            Glaube   
                    11                Universität        Hochschule 

In [14]:
file = join(DATA_BASE, 'topics_labels.csv')
df.to_csv(file)

In [8]:
ul = Unlemmatizer()

Loading ../data/preprocessed/dewiki_phrases_lemmatization_map.pickle


In [15]:
df.loc[:, 'term0':'term9']

term0             term1  \
dataset             topic_idx                                         
OnlineParticipation 1                      Arbeit      Gesellschaft   
                    2                   Vorschlag    Bürgerhaushalt   
                    3                       Seite            Brücke   
                    4                        Köln             Stadt   
                    6                        Idee             Stadt   
                    7                     Nutzung         Ehrenfeld   
                    8                     Schüler         Schülerin   
                    11                    Gebäude             Miete   
                    13                 Schlagloch              Zone   
                    14                     Lösung          Eindruck   
                    15                   Haushalt       Investition   
                    16                       Bonn              Oper   
                    17                 Stadt_Köln           Mülheim   
                    19                     Radweg               Weg   
                    20                Deutschland              Welt   
                    21                Unternehmen             Kunst   
                    23                       Euro              Jahr   
                    25                      Stadt           Angebot   
                    26                       Hund         Kontrolle   
                    28                      Platz              Müll   
                    34                        EUR          Leistung   
                    35                 Moderation              Gruß   
                    37                        PKW        Verbindung   
                    38                    Beitrag              Dank   
                    42                       Kind            Schule   
                    49                     Verein              Jahr   
                    50                    Wohnung          Wohnraum   
                    54                        Ort           Vorbild   
                    55                   Programm               Job   
                    56                     Antrag            Schutz   
                    59              Beschilderung             Recht   
                    60                     Schild             Linie   
                    62                 Spielplatz    Straßenlaterne   
                    64                   Übergang     Verkehrsinsel   
                    65                    Bahnhof        Kölnstraße   
                    66              Zebrastreifen              Höhe   
                    68              Fahrradfahrer     Einbahnstraße   
                    72                       Geld           Meinung   
                    74                        Bad        Grundstück   
                    79                       Auto               Rad   
                    81                     Fläche              Frau   
                    82                       ÖPNV            Stunde   
                    83                        Rat        Verwaltung   
                    88                     Anfang            Winter   
                    89                       Bahn        Stellplatz   
                    90                  Radfahrer         Fußgänger   
                    92                        Bus       Haltestelle   
                    94                 Autofahrer         Radfahrer   
                    95                       Haus               Uhr   
                    96                   Richtung            Radweg   
                    97                     Gehweg              Park   
PoliticalSpeeches   0                Wissenschaft         Forschung   
                    5                       Musik             Abend   
                    7                    Religion            Glaube   
                    11                Universität        Hochschule 

In [16]:
file = join(DATA_BASE, 'topics_unlem.csv')
df_topics_unlem = ul.unlemmatize_topics(df.loc[:, 'term0':'term9'])
df_topics_unlem.to_csv(file)
df_topics_unlem

Loading ../data/preprocessed/OnlineParticipation_lemmatization_map.pickle
    Schlagloch -> Schlaglöcher
    Stadt_Köln -> Stadt Köln
    Hund -> Hunde
    Kind -> Kinder
    Wohnung -> Wohnungen
    Auto -> Autos
    Fläche -> Flächen
    Bus -> Busse
    Schlagloch -> Schlaglöcher
    Stadt_Köln -> Stadt Köln
    Hund -> Hunde
    Kind -> Kinder
    Wohnung -> Wohnungen
    Auto -> Autos
    Fläche -> Flächen
    Bus -> Busse
    Schülerin -> Schülerinnen
    Jahr -> Jahren
    Kontrolle -> Kontrollen
    Leistung -> Leistungen
    Gruß -> Grüße
    Schule -> Schulen
    Jahr -> Jahren
    Frau -> Frauen
    Stunde -> Stunden
    Stellplatz -> Stellplätze
    Grünfläche -> Grünflächen
    Million -> Millionen
    Mensch -> Menschen
    Bank -> Bänke
    Student -> Studenten
    Gedanke -> Gedanken
    Jahr -> Jahren
    Gleis -> Gleise
    Tonne -> Tonnen
    Aussage -> Aussagen
    Familie -> Familien
    Veranstaltung -> Veranstaltungen
    Schuld -> Schulden
    Flüchtling -> Flüc

    Haar -> Haare
    Immobilie -> Immobilien
    Deutsch -> Deutschen
    Aktie -> Aktien
    Kunde -> Kunden
    Mensch -> Menschen
    Tote -> Toten
    Polizist -> Polizisten
    Mensch -> Menschen
    Produkt -> Produkte
    Tourist -> Touristen
    Temperatur -> Temperaturen
    Mensch -> Menschen
    Information -> Informationen
    Angestellte -> Angestellten
    Fan -> Fans
    Kandidat -> Kandidaten
    Abgeordnete -> Abgeordneten
    Minute -> Minuten
    Mensch -> Menschen
    Hesse -> Hessen
    deutsch_Bank -> Deutsche Bank
    Mensch -> Menschen
    Übergriff -> Übergriffe
    Monat -> Monaten
    Angabe -> Angaben
    Elter -> Eltern
    von_der_Leyen -> Von der Leyen
    Arbeitsplatz -> Arbeitsplätze
    Bundesland -> Bundesländern
    Migrant -> Migranten
    Million -> Millionen
    Zins -> Zinsen
    Soldat -> Soldaten
    Passagier -> Passagiere
    Filiale -> Filialen
    Medikament -> Medikamente
    Fotograf -> Fotografen
    Gerät -> Geräte
    Media -> Medien


term0              term1  \
dataset             topic_idx                                            
OnlineParticipation 1                        Arbeit       Gesellschaft   
                    2                     Vorschlag     Bürgerhaushalt   
                    3                         Seite             Brücke   
                    4                          Köln              Stadt   
                    6                          Idee              Stadt   
                    7                       Nutzung          Ehrenfeld   
                    8                       Schüler       Schülerinnen   
                    11                      Gebäude              Miete   
                    13                 Schlaglöcher               Zone   
                    14                       Lösung           Eindruck   
                    15                     Haushalt        Investition   
                    16                         Bonn               Oper   
                    17                   Stadt Köln            Mülheim   
                    19                       Radweg                Weg   
                    20                  Deutschland               Welt   
                    21                  Unternehmen              Kunst   
                    23                         Euro             Jahren   
                    25                        Stadt            Angebot   
                    26                        Hunde         Kontrollen   
                    28                        Platz               Müll   
                    34                          EUR         Leistungen   
                    35                   Moderation              Grüße   
                    37                          PKW         Verbindung   
                    38                      Beitrag               Dank   
                    42                       Kinder            Schulen   
                    49                       Verein             Jahren   
                    50                    Wohnungen           Wohnraum   
                    54                          Ort            Vorbild   
                    55                     Programm                Job   
                    56                       Antrag             Schutz   
                    59                Beschilderung              Recht   
                    60                       Schild              Linie   
                    62                   Spielplatz     Straßenlaterne   
                    64                     Übergang      Verkehrsinsel   
                    65                      Bahnhof         Kölnstraße   
                    66                Zebrastreifen               Höhe   
                    68                Fahrradfahrer      Einbahnstraße   
                    72                         Geld            Meinung   
                    74                          Bad         Grundstück   
                    79                        Autos                Rad   
                    81                      Flächen             Frauen   
                    82                         ÖPNV            Stunden   
                    83                          Rat         Verwaltung   
                    88                       Anfang             Winter   
                    89                         Bahn        Stellplätze   
                    90                    Radfahrer          Fußgänger   
                    92                        Busse        Haltestelle   
                    94                   Autofahrer          Radfahrer   
                    95                         Haus                Uhr   
                    96                     Richtung             Radweg   
                    97                       Gehweg               Park   
PoliticalSpeeches   0                  Wissenschaft          Forschung   
                    5                         Musi

In [17]:
file = join(DATA_BASE, 'labels_unlem.csv')
df_labels_unlem = ul.unlemmatize_labels(df.loc[:, 'label0':'label19'])
df_labels_unlem.to_csv(file)
df_labels_unlem

    öffentlich_Hand -> Öffentliche Hand
    Nationalstraße_B4 -> Nationalstraße B4
    franzen_Anton_Ries -> Franz Anton Ries
    Stadtwerk_Leipzig -> Stadtwerke Leipzig
    zweispurig -> Zweispurig
    sozial_Einrichtung -> Soziale Einrichtung
    Kanton_la_Tour-du-Pin -> Kanton La Tour-du-Pin
    da -> Da
    immer -> Immer
    Sonntagabend -> SonntagAbend
    Radfernweg_thüringer_Städtekette -> Radfernweg Thüringer Städtekette
    Aspekt_Salzburg -> Aspekte Salzburg
    Arbeitskreis_selbständig_Kultur-Institut -> Arbeitskreis selbständiger Kultur-Institute
    frei_Entfaltung_der_Persönlichkeit -> Freie Entfaltung der Persönlichkeit
    Regierungskonsultation -> Regierungskonsultationen
    israelitisch_Kultusgemeinde -> Israelitische Kultusgemeinde
    zeitgenössisch_Kunst -> Zeitgenössische Kunst
    visuell_Anthropologie -> Visuelle Anthropologie
    sozial_Engagement -> Soziales Engagement
    Menschenrecht -> Menschenrechte
    elektronisch_Media -> Elektronische Medien
    jun

    Lied -> Lieder
    heute_mittag -> Heute mittag
    Die_Polizei -> Die Polizei
    Android_TV -> Android TV
    Tall_Abyad -> Tall Abyad
    Million -> Millionen
    öffentlich_Haushalt -> Öffentlicher Haushalt
    Bundesstraße_482 -> Bundesstraße 482
    öffentlich_Bibliothek -> Öffentliche Bibliothek
    Staatsausgabe -> Staatsausgaben
    Communauté_D’Agglomération_de_Lens-Liévin -> Communauté d’agglomération de Lens-Liévin
    Staatseinnahme -> Staatseinnahmen
    öffentlich_Ruhe -> Öffentliche Ruhe
    technisch_Universität -> Technische Universität
    Kanton_Les_Deux_Rives -> Kanton Les Deux Rives
    sozial_Einrichtung -> Soziale Einrichtung
    mäßig_Geschwindigkeit -> Mäßige Geschwindigkeit
    Bilker_Allee -> Bilker Allee
    Einkunft -> Einkünfte
    Hollerstaude -> Hollerstauden
    urban_Landwirtschaft -> Urbane Landwirtschaft
    Vorsitzende -> Vorsitzender
    wohl -> Wohl
    Lehrschwimmbeck -> Lehrschwimmbecken
    Staatsstraße_2308 -> Staatsstraße 2308
    bayeri

    Fachhochschule_für_Finanz -> Fachhochschule für Finanzen
    Farn -> Farne
    zweite_Weltkrieg -> Zweiter Weltkrieg
    Die_Sonne -> Die Sonne
    Sonnenstern -> Sonnensterne
    sofortig_Beschwerde -> Sofortige Beschwerde
    Platzierungsrund -> Platzierungsrunde
    Angewandte_Kunst -> Angewandte Kunst
    Hauptstraße_34 -> Hauptstraße 34
    Haus_für_Kind -> Haus für Kinder
    Guilherand-Grang -> Guilherand-Granges
    Saint_Louis_Art_Museum -> Saint Louis Art Museum
    kosmisch_Staub -> Kosmischer Staub
    Die_Mutter -> Die Mutter
    Altria_Group -> Altria Group
    CIA -> Cia
    Maude_Adam -> Maude Adams
    Joch_Bendel -> Jochen Bendel
    Klaus_Höhne -> Klaus Höhne
    irakisch_Regierung -> Irakische Regierung
    historisch_Tatsache -> Historische Tatsachen
    islamisch_Emirat_Afghanistan -> Islamisches Emirat Afghanistan
    klein_Haus -> Kleines Haus
    Immobilienfond -> Immobilienfonds
    Full_Force -> Full Force
    häuslich_Gewalt -> Häusliche Gewalt
    Das_E

label0  \
dataset             topic_idx                                                  
OnlineParticipation 1                                       Öffentliche Hand   
                    2                                        Kommunalpolitik   
                    3                                          Straßenbrücke   
                    4                                                 Wetken   
                    6                                             Centre-Sud   
                    7                                             Freifläche   
                    8                                                Vorkurs   
                    11                                             Immobilie   
                    13                                     Nationalstraße B4   
                    14                                            Konsequenz   
                    15                                           Folgekosten   
                    16                                      Franz Anton Ries   
                    17                                Kornelimünster/Walheim   
                    19                                       Fußgängerbrücke   
                    20                                              Galerist   
                    21                                    Stadtwerke Leipzig   
                    23                                       Steueraufkommen   
                    25                                    Verkehrsverbindung   
                    26                                        Polizeibeamter   
                    28                                            Freifläche   
                    34                                                  Euro   
                    35                                             Bemerkung   
                    37                                            Zweispurig   
                    38                                                 Frage   
                    42                                             Vorschule   
                    49                                            Freifläche   
                    50                                     Studentenwohnheim   
                    54                                   Soziale Einrichtung   
                    55                                 Kanton La Tour-du-Pin   
                    56                                         Zusammenleben   
                    59                                       Verkehrsführung   
                    60                                       Haltestellenkap   
                    62                                            Freifläche   
                    64                                           Hauptstraße   
                    65                                              Ringbahn   
                    66                                       Straßenkreuzung   
                    68                                         Einbahnstraße   
                    72                                                    Da   
                    74                                    Naherholungsgebiet   
                    79                                         Stadtautobahn   
                    81                                            Freifläche   
                    82                                            Nahverkehr   
                    83                                           Gemeinderat   
                    88                                                 Immer   
                    89                                            Sportplatz   
                    90                                                 Ampel   
                    92                                        Verkehrsmittel   
                    94                                       Kraftfahrstraße   
                    95                                          Sonnta